# Data Collection For Nowcasting Thai GDP

This notebook documents the data acquisition process and contains the complete Python script used to retrieve the relevant datasets. The primary objective is to collect 16 monthly macroeconomic time series from the Bank of Thailand (BOT) database and one monthly series for the SET index from a CSV file downloaded from the Stock Exchange of Thailand (SET) website. These series will serve as predictors for the nowcasting of Thailand's GDP. The GDP data itself is sourced separately from the National Economic and Social Development Council (NESDC) website and loaded from an exported XLSX file.

In the final step, data vintages are constructed to simulate a real-time forecasting environment. These prepared datasets are then exported for use in the subsequent preprocessing, modeling and evaluation stages. Additional details regarding the choice of indicators are discussed in the data chapter of the accompanying thesis.

In [1]:
# Import required libraries
import pandas as pd
import http.client
import json
import requests
import time
import re
import numpy as np

## Get Data From Bank of Thailand (BOT)

In [2]:
# Load the Excel file containing multiple sheets
file_path = "./API_Statistic_time_series.xlsx"
bot_api_stats = pd.read_excel(file_path, sheet_name=None)

# Initialize a list to hold all DataFrames from each sheet
all_dfs = [] 

# Iterate through each sheet in the workbook
for sheet_name, df in bot_api_stats.items():
    df.columns = df.iloc[0]  # Set the first row as column headers 
    df = df[1:].reset_index(drop=True) # Remove the header row from data  
    df["category"] = sheet_name # Add a column to identify the source sheet
    all_dfs.append(df)

# Combine all DataFrames into a single DataFrame
bot_db_stats = pd.concat(all_dfs, ignore_index=True)

# Clean column names for consistency and easier manipulation
bot_db_stats.columns = (
    bot_db_stats.columns.str.strip() # Remove leading/trailing spaces
    .str.replace(r"\s+", "_", regex=True) # Replace internal spaces with underscores
    .str.replace(r"[^\w\s]", "", regex=True) # Remove special characters
    .str.lower() # Convert all column names to lowercase
)

In [3]:
# Display the "bot_db_stats" DataFrame info
bot_db_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648 entries, 0 to 27647
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   report_code_bot_website  27648 non-null  object
 1   data_series_code         27648 non-null  object
 2   data_series_name_th      27648 non-null  object
 3   data_series_name_eng     27648 non-null  object
 4   unit                     27206 non-null  object
 5   frequency                27616 non-null  object
 6   data_type                27616 non-null  object
 7   description_th           27615 non-null  object
 8   description_eng          27616 non-null  object
 9   lag_time                 27590 non-null  object
 10  release_schedule_th      27616 non-null  object
 11  release_schedule_eng     27616 non-null  object
 12  source_of_data_th        27616 non-null  object
 13  source_of_data_eng       27595 non-null  object
 14  observation_start        27616 non-nul

In [4]:
# Count the number of records by frequency type
frequency_counts = bot_db_stats["frequency"].value_counts()

# Calculate the percentage of each frequency
frequency_percentage = (frequency_counts / len(bot_db_stats)) * 100

# Combine counts and percentages into a summary table
frequency_summary = pd.DataFrame({
    "Count": frequency_counts,
    "Percentage": frequency_percentage.round(2)
})

# Display frequency distribution summary
print(frequency_summary)

             Count  Percentage
frequency                     
Quarterly    18628       67.38
Monthly       7597       27.48
Annual        1010        3.65
M,Q,Y          332        1.20
Daily           19        0.07
Semi-annual     17        0.06
Weekly          12        0.04
Fortnightly      1        0.00


In [5]:
# Filter dataset to include only monthly frequency data
bot_db_stats_filt = bot_db_stats[bot_db_stats["frequency"].isin(["Monthly"])] 

In [6]:
# Manually preselect series codes of interest
presel_series_code = ['PFCG000000M000651', 'EICEIM00002', 'EILEIM00011',
       'EIPCIM00063', 'EIPIIM00052', 'EIBSIM00060', 'EINEERM00071',
       'EIREERM00072', 'EIEXTHBM00152', 'EIIMTHBM00155',
       'EITRADERETM00384', 'EITRADERETM00391', 'EITRADEWHOM00426', 'EISPIM00511',
    'EISPIM00507', 'EIBSIOTHM00445']

In [7]:
# Further filter the economic indicators to only include 
# the variables in the 'variables' list
bot_db_stats_filt_sub = bot_db_stats_filt[bot_db_stats_filt['data_series_code']\
.isin(presel_series_code)]

In [8]:
# Convert to datetime format
parsed_dates = pd.to_datetime(bot_db_stats_filt_sub["observation_start"], 
                              errors="coerce", dayfirst=True)

# Count the frequency of each monthly start date
monthly_start_freq = parsed_dates.value_counts(normalize=True).sort_index() * 100

# Calculate the cumulative percentage
cumulative_percentage = monthly_start_freq.cumsum()

# Print the results
print(f"Cumulative percentage of the observation starts by month:\n{cumulative_percentage}")

Cumulative percentage of the observation starts by month:
observation_start
1994-01-01     12.50
1999-01-01     18.75
2000-01-01     62.50
2008-01-01     68.75
2009-01-01     75.00
2010-01-01     87.50
2011-01-01    100.00
Name: proportion, dtype: float64


In [9]:
# Define function to fetch observations
def fetch_observations(series_code, start_period, end_period):
    """
    Fetches time series observations from the Bank of Thailand (BOT) API.

    This function retrieves a specified macroeconomic time series from the BOT API 
    within a given date range. It includes built-in handling for API authentication, 
    automatic retry on rate limiting (HTTP 429) and response parsing into a clean 
    pandas DataFrame.

    Parameters
    ----------
    series_code : str
        The BOT series code identifying the macroeconomic indicator to fetch.
    start_period : str
        The start date for the query in 'YYYY-MM-DD' format.
    end_period : str
        The end date for the query in 'YYYY-MM-DD' format.

    Returns
    -------
    pd.DataFrame or None
        A DataFrame containing the retrieved observations with columns:
        ['series_code', 'series_name', 'date', 'value'].
        Returns None if no data is found or if an error occurs.

    Notes
    -----
    - If the API rate limit is exceeded (status code 429), 
        the function waits 60 seconds and retries.
    - API responses are expected to follow the BOT's public API format.
    - Requires an active internet connection and valid BOT API credentials.
    """

    # Define URL and headers
    url = f"https://apigw1.bot.or.th/bot/public/observations/?series_code={series_code}&start_period={start_period}&end_period={end_period}"
    headers = {
        'X-IBM-Client-Id': "insert-your-api-key", # Insert your API key from BOT
        'accept': "application/json"
    }

    try:
        # Send the GET request
        response = requests.get(url, headers=headers)
        
        # Check for rate limiting
        if response.status_code == 429:
            print("Rate limit exceeded. Sleeping for 60 seconds...")
            time.sleep(60)  # Wait for 60 seconds before retrying
            return fetch_observations(series_code, 
                                      start_period, end_period)  # Retry the request
        
        response.raise_for_status()  # Raise exception for 4xx or 5xx errors
        data = response.json()

        # Extract observations if present
        if "result" in data and "series" in data["result"]:
            series_data = data["result"]["series"][0]  # Extract first series entry

            observations = series_data.get("observations", [])
            if not observations:
                print(f"No observations found for {series_code}.")
                return None  # Return None if there are no observations

            # Process the observations into a DataFrame
            records = [
                {
                    "series_code": series_data["series_code"],
                    "series_name": series_data["series_name_eng"],
                    "date": obs["period_start"],
                    "value": obs["value"]
                }
                for obs in observations
            ]
            
            # Convert to DataFrame
            return pd.DataFrame(records)
        
        else:
            print(f"No data found for {series_code}")
            return None
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {series_code}: {e}")
        return None

In [10]:
# Initialize a list to store all valid series' fetched data
all_valid_series_data = []

# Ensure proper column naming
bot_db_stats_filt_sub = bot_db_stats_filt_sub\
.rename(columns={"data_series_code": "series_code"})

# Loop through all unique series codes and fetch their observations
for series_code in bot_db_stats_filt_sub.series_code.unique():
    try:
        # Try fetching observations
        result = fetch_observations(
            series_code, 
            start_period="2011-01-01", 
            end_period="2019-12-31"
        )
        
        # Check if result is not None and is a DataFrame
        if result is not None and not result.empty:
            # Optionally, add the series_code as a column to track source
            result['series_code'] = series_code
            # Append to list
            all_valid_series_data.append(result)

    except Exception as e:
        print(f"Failed to fetch data for {series_code}: {e}")

# Combine into a single DataFrame
if all_valid_series_data:
    bot_db_stats_final = pd.concat(all_valid_series_data, ignore_index=True)
else:
    bot_db_stats_final = pd.DataFrame()  # Empty fallback

In [11]:
# Merge metadata columns (data_type, lag_time, release_schedule) into final dataset
bot_db_stats_final = bot_db_stats_final.merge(
    bot_db_stats_filt_sub[["series_code", "data_type", 
                           "lag_time", "release_schedule_eng"]],
    on="series_code",
    how="left"
)

In [12]:
# Define functions for getting release dates for Vintage creation

# Import library for date offsetting
import pandas.tseries.offsets as offsets

# Load the Thai Financial Holidays
thai_hol = pd.read_csv("thai-financial-institutions-holidays.csv")

# Convert holiday 'date' column to datetime data type
thai_hol['date'] = pd.to_datetime(thai_hol['date'], format="%d-%m-%Y")

# Get holiday dates as a set for fast lookup
holiday_dates = set(thai_hol['date'].dt.date)

# Convert 'date' in main dataset to datetime and set to month end
bot_db_stats_final['date'] = pd.to_datetime(bot_db_stats_final['date'], 
                                            format='%Y-%m') + pd.offsets.MonthEnd(0)


# Define the holiday and weekend adjustment function
def adjust_for_holiday_and_weekend(rel_date, holiday_dates, 
                                   is_first_business_day=False):
    """
    Adjusts a release date to the nearest valid business day.
    
    If `is_first_business_day=True`, moves forward to the next business day.
    Otherwise, moves backward to the most recent business day.
    
    Parameters
    ----------
    rel_date: Initial release date
    holiday_dates: Set of public holidays
    is_first_business_day: Whether to move forward or backward
    
    Returns
    -------
    Adjusted business day (datetime)
    """
    while True:
        is_holiday = rel_date.date() in holiday_dates
        is_weekend = rel_date.weekday() >= 5  # 5 = Saturday, 6 = Sunday

        if not is_holiday and not is_weekend:
            break

        if is_first_business_day:
            # Move forward to the next business day for the first business day
            rel_date += offsets.BDay(1)
        else:
            # Move backward to the previous business day for other cases
            rel_date -= offsets.BDay(1)

    return rel_date

# Define function for calculating the release dates
def calculate_release_date(row, holiday_dates):
    """
    Calculates the release date for a given time series record.

    This function determines the actual release date based on the reference period,
    the specified release schedule (e.g., first or last business day of a future month),
    and any additional lag. It also adjusts for Thai financial holidays and weekends
    to ensure the release date falls on the correct business day.

    Parameters
    ----------
    row : pd.Series
        A row from a DataFrame representing a time series record. Must contain:
        - 'date': pd.Timestamp indicating the end of the reference month.
        - 'release_schedule_eng': str describing the release schedule in English.
        - 'lag_time': str or None, indicating lag (e.g., '1 month', '2 months').
    holiday_dates : list of datetime.date
        A list of dates representing Thai financial market holidays. Used to
        adjust the release date to the nearest valid business day.

    Returns
    -------
    pd.Timestamp or pd.NaT
        The adjusted release date that complies with the specified schedule and
        avoids holidays and weekends. Returns `pd.NaT` if the schedule is
        unrecognized or improperly specified.

    Notes
    -----
    - The function supports common schedule descriptions such as:
        * "The first business day of the following month"
        * "The last business day of the following month"
        * "The last business day of the second month after the reference period"
    - Unknown or unsupported schedule strings will result in `pd.NaT`.
    - Lag time is interpreted in months and defaults to one month if ambiguous.

    See Also
    --------
    adjust_for_holiday_and_weekend : Helper function used to align dates with business days.
    """
    ref_date = row['date']
    schedule = row['release_schedule_eng']
    lag = str(row.get('lag_time'))\
    .lower() if pd.notnull(row.get('lag_time')) else None

    if schedule == 'The first business day of the following month':
        # Determine lag: default = no lag => next month's first business day
        if lag == '2 month' or lag == '2 months':
            ref_date += offsets.MonthEnd(2)
        elif lag == '1 month':
            ref_date += offsets.MonthEnd(1)
        # if lag is None (no lag), use ref_date as-is
        
        # Move to first business day **after** that date
        rel_date = ref_date + offsets.BDay(1)
        rel_date = adjust_for_holiday_and_weekend(rel_date, holiday_dates, 
                                                  is_first_business_day=True)

    elif schedule == 'The last business day of the second month after the reference period':
        ref_date += offsets.MonthEnd(2)
        rel_date = adjust_for_holiday_and_weekend(ref_date, holiday_dates, 
                                                  is_first_business_day=False)

    elif schedule in ['The last business day of the following month',
                      'The last business day of the next month',
                      'The last business day']:
        if lag == '1 month':
            ref_date += offsets.MonthEnd(1)
        elif lag is None:
            pass  # use ref_date directly
        else:
            ref_date += offsets.MonthEnd(1)  # fallback to 1 month for other/unexpected lags

        rel_date = adjust_for_holiday_and_weekend(ref_date, holiday_dates, 
                                                  is_first_business_day=False)

    else:
        return pd.NaT  # Unknown schedule

    return rel_date

In [13]:
# Apply the release date calculation to each row
bot_db_stats_final['release_date'] = bot_db_stats_final.apply(
    lambda row: calculate_release_date(row, holiday_dates), axis=1
)

In [14]:
# Check for duplicate date entries within each series
duplicates = bot_db_stats_final[bot_db_stats_final.duplicated(subset=['series_code', 
                                                                      'date'], 
                                                              keep=False)]

# Display duplicates if found
if not duplicates.empty:
    print(f"Found {len(duplicates)} duplicate rows with the same series_code and date.")
else:
    print("No duplicate (series_code, date) combinations found in the final dataset.")

No duplicate (series_code, date) combinations found in the final dataset.


In [15]:
# Check info before exporting
bot_db_stats_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   series_code           1728 non-null   object        
 1   series_name           1728 non-null   object        
 2   date                  1728 non-null   datetime64[ns]
 3   value                 1728 non-null   object        
 4   data_type             1728 non-null   object        
 5   lag_time              1512 non-null   object        
 6   release_schedule_eng  1728 non-null   object        
 7   release_date          1728 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(6)
memory usage: 108.1+ KB


In [16]:
# Export BOT dataset to csv
# Uncomment if needed
#bot_db_stats_final.to_csv("bot_dataset.csv", index=False)

## Get Financial Data From SET

In [17]:
# Load SET Index data
set_df = pd.read_csv("set_index.csv", skiprows=3)

# Drop any rows where the first column is "Base" (unit information)
set_df = set_df[set_df.iloc[:, 0] != "Base"]

# Select only the first two columns (Date and Value) and limit to expected row count
set_df = set_df.iloc[:601, [0, 1]]

# Rename columns to more meaningful names
set_df.columns = ["date_raw", "value_raw"]

# Convert to datetime format with end-of-month alignment
set_df["date"] = pd.to_datetime(set_df["date_raw"], format="%b-%Y", errors="coerce")
set_df["date"] = set_df["date"] + pd.offsets.MonthEnd(0)

# Convert the value column to numeric, removing commas
set_df["value"] = pd.to_numeric(set_df["value_raw"].astype(str).str.replace(",", ""), 
                                errors="coerce")

# Add required columns to match bot_db_stats_final
set_df["series_code"] = "SETINDEXTHA"
set_df["series_name"] = "SET Index"
set_df["data_type"] = "Index"
set_df["lag_time"] = np.nan
set_df["release_schedule_eng"] = np.nan
set_df["release_date"] = pd.NaT

# Keep only necessary columns
df_set_final = set_df[[
    "series_code", "series_name", "date", "value",
    "data_type", "lag_time", "release_schedule_eng", "release_date"
]]

# Drop rows with missing dates or values
df_set_final = df_set_final.dropna(subset=["date", "value"])

# Filter the period from 2011Q1 to 2019Q4
df_set_final = df_set_final[df_set_final["date"].between("2011-01-01", "2019-12-31")]

# Reindex to ensure complete end-of-month frequency using 'ME'
full_index = pd.date_range(start="2011-01-31", end="2019-12-31", freq="ME")
df_set_final = df_set_final.set_index("date").reindex(full_index)

# Set frequency explicitly for compatibility with X-13
df_set_final.index.freq = "ME"

# Restore date column
df_set_final = df_set_final.reset_index().rename(columns={"index": "date"})

In [18]:
# Check df_set_final info
df_set_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  108 non-null    datetime64[ns]
 1   series_code           108 non-null    object        
 2   series_name           108 non-null    object        
 3   value                 108 non-null    float64       
 4   data_type             108 non-null    object        
 5   lag_time              0 non-null      float64       
 6   release_schedule_eng  0 non-null      float64       
 7   release_date          0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(3)
memory usage: 6.9+ KB


In [19]:
# Append the data to get final dataset
final_df = pd.concat([bot_db_stats_final, df_set_final], ignore_index=True)

In [20]:
# Export final dataset (information set)
# Uncomment if needed
#final_df.to_csv("set_bot_dataset_final.csv", index=False)

## Get Quarterly GDP Growth Rate from NESDC

In [21]:
# Load the NESDC Excel file
file_path = "./article_file_20250217085437.xlsx"
nesdc_gdp = pd.read_excel(file_path, sheet_name="Table 2.2", skiprows=8)

In [22]:
# Extract the year (4-digit number at the start of the string), 
# even with suffix like '2021r'
nesdc_gdp['Year'] = (
    nesdc_gdp['Unnamed: 0']
    .astype(str)
    .str.extract(r'(?P<year>\d{4})')['year']
    .ffill()
    .astype(int)
)

# Filter rows that represent quarters (e.g., 'Q1', 'Q2', etc.)
quarter_mask = nesdc_gdp['Unnamed: 0'].astype(str).str.contains(r'^Q[1-4]', na=False)
quarterly_gdp = nesdc_gdp[quarter_mask].copy()

# Extract the quarter from the 'Unnamed: 0' column, 
# assuming it's in the format Q1, Q2, etc.
quarterly_gdp['Quarter'] = quarterly_gdp['Unnamed: 0'].astype(str)\
.str.extract(r'(Q[1-4])')[0]

# Combine year and quarter into 'YYYYQx' format
quarterly_gdp['quarter'] = quarterly_gdp['Year']\
.astype(str) + quarterly_gdp['Quarter']

# Select and rename the GDP growth column
quarterly_gdp_final = quarterly_gdp[['(11)', 'quarter']].copy()
quarterly_gdp_final.rename(columns={'(11)': 'gdp_growth'}, inplace=True)

# Filter for the range from 2011-Q1 to 2019-Q4
quarterly_gdp_final = quarterly_gdp_final[
    (quarterly_gdp_final['quarter'] >= '2011Q1') & 
    (quarterly_gdp_final['quarter'] <= '2019Q4')
].reset_index(drop=True)

In [23]:
# Replace full-revised 2019Q4 GDP data with the preliminary one
# Source: https://www.nesdc.go.th/nesdb_en/article_attach/article_file_20200221170629.pdf
quarterly_gdp_final.loc[quarterly_gdp_final.index[-1], 'gdp_growth'] = 1.6

In [24]:
# Show final dataframe info
quarterly_gdp_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gdp_growth  36 non-null     float64
 1   quarter     36 non-null     object 
dtypes: float64(1), object(1)
memory usage: 704.0+ bytes


In [25]:
# Export cleaned target data
# Uncomment if needed
#quarterly_gdp_final.to_csv("thai-gdp-quarterly-2011Q1-2019Q4.csv", index=False)